![Assimila logo](../helpers/logosmall.png "Assimila")
# Assimila Prise Products
# Analyse Rainfall Product

If the drop down menus do not appear restart the notebook by selecting "Kernel" then "Restart" above.

In [51]:
import os
import sys

sys.path.append('../')
from DQTools.DQTools import Dataset

sys.path.append('../helpers/')
from widgets import Widgets
from data import Data

import matplotlib

import matplotlib.cm as cm
import matplotlib.animation as animation

import matplotlib.cbook as cbook
import matplotlib.image as image

matplotlib.use('nbagg')
import matplotlib.pyplot as plt

import gdal
import numpy as np
import xarray as xr

import ipywidgets as widgets
import pandas as pd
import datetime
import calendar

from IPython.display import clear_output
from IPython.display import FileLink


from IPython.core.display import display, HTML

# Change display width of the notebook to 100%
display(HTML("<style>.container { width:100% !important; }</style>"))
# Stops widget labels from being cut off by widget
display(HTML("<style>.widget-label { min-width: 20ex !important; }</style>"))

%matplotlib notebook 

def analyse(val, timeseries=False):
    """
    Handle the DQTools request and calculate the mean of the data for a monthly/yearly timestep.
    
    :returns: DataArray of monthly/yearly averaged data.
    """ 
    if timestep.value == "months":
        # Number of days in the user specified month
        num_days = calendar.monthrange(val.year, val.month)[1]

        # Get a list of dates between in the specified month
        days = [datetime.date(val.year, val.month, day) for day in range(1, num_days+1)]
    
    elif timestep.value == "years":
        days = []
        months = ["01", "02", "03", "04", "05", "06", 
                  "07", "08", "09", "10", "11", "12"]
        
        for month in months:
            num_days = calendar.monthrange(val.year, int(month))[1]
            for day in range(1, num_days+1):
                days.append(datetime.date(val.year, int(month), day))
    
    if timeseries:
        # Query DQTools for the selected month at the specified lat/lon
        rfe_data = get_data(product='chirps', subproduct='rfe', start=days[0], stop=days[-1],
                            latlon=[lat.value, lon.value])
    else:
        # Query DQTools for the selected month and chip area
        rfe_data = get_data(product='chirps', subproduct='rfe', start=days[0], stop=days[-1],
                            region=[lat.value+1.5, lon.value+1.5, lat.value-1.5, lon.value-1.5])
        
    return rfe_data.mean('time')


def on_dropdown_dates_change(*args, add_colorbar=False):
    global rfe_data, data
    """
    Handles change event on dropdown_dates widget
    """
    if args:
        val = args[0]['new']
    else:
        val = dropdown_dates.value
    
    if val is None:
        return
    
    rfe_chip_p.set_visible(False)
    
    # Delete last reference point
    if len(rfe_p.lines) > 0:
        del rfe_p.lines[0]
    
    # Display a full Africa color map if a single day is chosen (fast query)
    if timestep.value == "days":
        # Query DQTools for user selected dropdown date and select it
        rfe_data = get_data(product='chirps', subproduct='rfe', start=val, stop=val)
        data = rfe_data.sel(time=dropdown_dates.value)
        
        colorbars = []
        rfe_p.clear()
        ts_p.clear()
        tmp_img = data.plot.imshow(ax=rfe_p, cmap='Blues', add_colorbar=False)

        rfe_p.set_title('rfe')
        rfe_p.set_aspect('equal')
        if add_colorbar:
            colorbars.append(fig.colorbar(tmp_img, ax=rfe_p, orientation="horizontal", fraction=0.044, pad=0.1))
        rfe_p.grid()

        with out:
            fig.canvas.draw()
            plt.show()
    
    elif timestep.value == "months":
       # Plot monthly average
        monthly_data = analyse(dropdown_dates.value)
        monthly_data.plot(ax=rfe_chip_p, cmap='Blues', add_colorbar=False)
        rfe_chip_p.set_title(f'time = {dropdown_dates.value.year}-{dropdown_dates.value.month}')
        rfe_chip_p.grid()
        rfe_chip_p.set_visible(True)
        

def on_click(event):
    """
    Event handler
    """
    return
    # Event does not apply for time series plot
    # Check if the click was in a
    ts_p.clear()

    # Delete last reference point
    if len(rfe_p.lines) > 0:
        del rfe_p.lines[0]

    # Draw a point as a reference
    rfe_p.plot(event.xdata, event.ydata,
            marker='o', color='red', markersize=7, alpha=0.7) 
    
    # Define a 'chip' of data to display to the user (9 square degrees)
    rfe_chip = rfe_data.where((rfe_data["latitude"] <lat.value+1.5)&
                              (rfe_data["latitude"] >lat.value-1.5)&
                              (rfe_data["longitude"]<lon.value+1.5)&
                              (rfe_data["longitude"]>lon.value-1.5), drop=True)
    
    # Plot the chip of data
    tmp_img = rfe_chip.plot(ax=rfe_chip_p, cmap='Blues', add_colorbar=False)
    rfe_chip_p.grid()
    rfe_chip_p.set_visible(True)
    fig.canvas.draw()
    
def on_button_clicked(b):
    """
    Event handler function for 'Get location'
    """ 
    # Clear the timeseries plot
    ts_p.clear()
    
    # Delete last reference point
    if len(rfe_p.lines) > 0:
        del rfe_p.lines[0]
    
    # Draw a point as a reference on both maps
    rfe_p.plot(lon.value, lat.value,
            marker='o', color='red', markersize=7, alpha=0.7) 
    rfe_chip_p.plot(lon.value, lat.value,
            marker='o', color='red', markersize=7, alpha=0.7) 
    
    # Plot the chip of data if a daily timestep is chosen
    if timestep.value == 'days':
        # Define a 'chip' of data to display to the user (9 square degrees)
        rfe_chip = data.where((rfe_data["latitude"] <lat.value+1.5)&
                                  (rfe_data["latitude"] >lat.value-1.5)&
                                  (rfe_data["longitude"]<lon.value+1.5)&
                                  (rfe_data["longitude"]>lon.value-1.5), drop=True)
        
        # Plot
        tmp_img = rfe_chip.plot(ax=rfe_chip_p, cmap='Blues', add_colorbar=False)
        rfe_chip_p.grid()
        rfe_chip_p.set_visible(True)
    
    elif timestep.value == 'months':
        # Plot monthly average
        monthly_data = analyse(dropdown_dates.value)
        monthly_data.plot(ax=rfe_chip_p, cmap='Blues', add_colorbar=False)
        rfe_chip_p.set_title(f'time = {dropdown_dates.value.year}-{dropdown_dates.value.month}')
        rfe_chip_p.grid()
        rfe_chip_p.set_visible(True)
        
    elif timestep.value == 'years':
        # Plot yearly average
        yearly_data = analyse(dropdown_dates.value)
        yearly_data.plot(ax=rfe_chip_p, cmap='Blues', add_colorbar=False)
        rfe_chip_p.set_title(f'time = {dropdown_dates.value.year}')
        rfe_chip_p.grid()
        rfe_chip_p.set_visible(True)

def on_timestep_change(*args):
    """
    Event handler function for changing timestep/start/end dates.
    """
    if args:
        val = args[0]['new']
    else:
        val = timestep.value
    
    # Clear the exisiting plots
    rfe_chip_p.set_visible(False)
    ts_p.clear()
    
    # Delete last reference point
    if len(rfe_p.lines) > 0:
        del rfe_p.lines[0]
    
    # Get a list of dates between the specified start and end date selected
    dates = [start.value+datetime.timedelta(days=x) for x in range((end.value-start.value).days)]
 
    if val == 'days':
        # Set the dropdown date options to daily increments
        dropdown_dates.options = dates
        
    elif val == 'months':
        # Set the dropdown date options to monthly increments
        _months = pd.date_range(dates[0], dates[-1], freq='MS')
        dropdown_dates.options = [pd.to_datetime(d).date() for d in _months]
        
    elif val == 'years':
        # Set the dropdown date options to yearly increments
        _years = pd.date_range(dates[0], dates[-1], freq='YS')
        dropdown_dates.options = [pd.to_datetime(d).date() for d in _years]
        
def on_date_change(*args):
    """
    Event handler function for changing start and end dates
    """ 
    on_timestep_change()

def on_data_button_clicked(b):
    """
    Event handler function for pressing 'Get data' button
    """
    button_data.disabled = True
    
    if len(rfe_p.lines) == 0:
        on_button_clicked(b)
    
    # Clear timeseries plot
    ts_p.clear()

    # Days
    if timestep.value == 'days':
        # Request data
        rfe_data = get_data(start=start.value, stop=end.value, latlon=[lat.value, lon.value])
        
        delta = end.value-start.value
        period = [start.value + datetime.timedelta(days=i) for i in range(delta.days)]

        _ts = [rfe_data.sel(time=day) for day in period]
         
    # Months
    if timestep.value == 'months':
        period = pd.date_range(start.value, end.value, freq='MS')
        _ts = [analyse(month, timeseries=True) for month in period]
    
    # Years
    if timestep.value == 'years':
        period = pd.date_range(start.value, end.value, freq='YS')
        _ts = [analyse(year, timeseries=True) for year in period]
        
    # Concatenate arrays
    ts = xr.concat([d for d in _ts], dim='time')

    # Redefine x-coordinate values 
    ts.coords['time']  = ('time', period)
    
    # Draw a point as a reference
    rfe_p.plot(lon.value, lat.value,
            marker='o', color='red', markersize=7, alpha=0.7)
    ts_p.plot(ts.time.data, ts.data, label=f'CHIRPS RFE',
                 color='blue', lw=1.5, alpha=0.5)
    ts_p.axhline(threshold.value, linestyle='--', color='red', label='threshold')
    ts_p.legend()
    ts_p.set_title('')
    tkw = dict(size=4, width=1.5)
    ts_p.tick_params(axis='y', **tkw)
    
    if timestep.value == 'daily':
        ts_p.set_ylabel('rfe (mm/day)')
    else:
        ts_p.set_ylabel('average rfe (mm/day)')
        
    ts_p.grid()
    
    with out:
        fig.canvas.draw() 
        
        print(f"""
=======================================================================================      
The rainfall threshold was exceeded {len(ts.where(ts>threshold.value, drop=True).data)} times over date range {start.value} to {end.value}
=======================================================================================          
        """)
        
        # Display a csv download link
        filename = f"chirps_rfe{lat.value}_{lon.value}" \
                   f"_{start.value}_{end.value}.csv"
        ts.to_dataframe().to_csv(filename)
        display(FileLink(filename))
        
    button_data.disabled = False

In [52]:
############
# Get data #
############

def send_request(product, subproduct, start, stop, region, latlon):
    """
    Send a request for data to the DataCube using DQTools
    """
    ds = Dataset(product=product, 
                 subproduct=subproduct,
                 identfile=os.path.join(os.path.expanduser("~"),
                                        '.assimila_dq.txt'))

    if latlon is None:
        ds.get_data(start=start, 
                    stop=stop,
                    region=region)
    else: 
        ds.get_data(start=start, 
                stop=stop,
                latlon=latlon)
    
    
    return ds.data

# TODO: change to Africa tile for initialisation and region for chip call 
def get_data(product='chirps', subproduct='rfe', start=datetime.datetime(2009, 1, 1), 
             stop=datetime.datetime(2009, 1, 1), region=[40, 55, -40, -20], latlon=None):
    """
    Call the DataCube request function above and get resultant data into correct
    format and extarct a list of times.
    """
    now = datetime.datetime.now()
    
    rfe_data = send_request(product, subproduct, start, stop, region, latlon)["rfe"]
#     with out:
#         print(f"Request runtime: {datetime.datetime.now()-now} seconds")
    #times = sorted(set([pd.to_datetime(d).date() for d in rfe_data.time.data]))
    
    return rfe_data

In [57]:
###########################
# Get initialisation data #
###########################

rfe_data = get_data()

###########
# Widgets #
###########

w = Widgets()

freq = ['days', 'months', 'years']

lat = w.latitude()
lon = w.longitude()
lat.value, lon.value = -22, 32
button = widgets.Button(description='Get point',
                        disabled=False,
                        button_style='info',
                        tooltip='Click me to plot location data')

button_data = widgets.Button(description='Get data',
                        disabled=False,
                        button_style='info',
                        tooltip='Click me to retrieve selected data') 

start = w.get_date(value=datetime.date(2009,1,1), description='Start')
end = w.get_date(value=datetime.date(2009,12,31), description='End')

threshold = widgets.BoundedFloatText(value=7.5,
                                    min=0,
                                    max=10.0,
                                    step=0.1,
                                    description='Threshold (mm/day):',
                                    disabled=False)

dropdown_dates = widgets.Dropdown(description='Date selection',
                                  disabled=False)

timestep = widgets.Dropdown(options=freq,
                            value=freq[0],
                            description='Timestep',
                            disabled=False)

output_layout = widgets.Layout(width='100%', height='1600px')
out = w.display_output()
out.layout = output_layout
    
button.on_click(on_button_clicked)
button_data.on_click(on_data_button_clicked)

data_list = dropdown_dates.observe(on_dropdown_dates_change, names='value')
# start.observe(on_date_change)
# end.observe(on_date_change)
timestep.observe(on_timestep_change)

box1 = widgets.HBox([lat, lon, button])
box2 = widgets.HBox([threshold])
box3 = widgets.HBox([start, end, timestep, dropdown_dates, button_data])
box = widgets.VBox([box1, box2, box3])

with out:
    display(box)
    
    ################
    # Plot objects #
    ################
    
    fig = plt.figure(figsize=(16, 11))

    rfe_p = plt.subplot2grid((2, 3), (0, 1), colspan=1)
    
    rfe_chip_p = plt.subplot2grid((2, 3), (0, 2), colspan=1)
    
    rfe_chip_p.set_visible(False)
    
    plt.subplots_adjust(wspace=0.13, hspace=0.1)

    ts_p = plt.subplot2grid((2, 3), (1, 0), colspan=3)

# Connect the canvas with the event
cid = fig.canvas.mpl_connect('button_press_event', on_click)

images = []
on_timestep_change()
on_dropdown_dates_change(add_colorbar=True)

Output()